# Data preprocessing

In [1]:
# Import libraries
import pandas as pd
import os
import csv
import billboard
import lyricsgenius
import json
import re

In [2]:
# Loading data and adding year columns and wrapping up billboard songs (1950-2015)
cwd = os.getcwd()
data_path = os.chdir(cwd + "\data")
all_csv_files = os.listdir(data_path)
songs_50_15 = []

for filename in all_csv_files:
    songs = pd.read_csv(filename, header = 0, sep = ';')#, error_bad_lines=False)
    year = os.path.basename(filename)[:-4]
    songs['Year'] = year
    songs_50_15.append(songs)

In [3]:
# Downloading data from years 2016-2020 from billboard library
playlist = 'hot-100-songs'
dates = [2016, 2017, 2018, 2019, 2020]
songs_billboard = []
for y in dates:
    songs = billboard.ChartData(playlist, date = None, year = y, fetch = True, timeout = 25)
    songs_billboard.append(songs)

In [4]:
# Preprocessing data from years 2016-2020
song_titles = []
artists = []
for year in songs_billboard:
    for playlist in year:
        song = str(playlist).split(" by ")
        song_titles.append(song[0].strip("\'"))
        artists.append(song[1])
        
# Adding missed song on 87th position in 2016 chart
song_titles.insert(86, 'All the Way Up')
artists.insert(86, 'Fat Joe, Remy Ma and Jay-Z featuring French Montana and Infared')

In [5]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Splitting list into same-length elements lists
def list_split(l, n):
    n = max(1, n)
    return (l[i:i+n] for i in range(0, len(l), n))

split_artists = list(list_split(artists, 100))
split_song_titles = list(list_split(song_titles, 100))


# Creating list of dictionaries for each year to convert them into pandas DataFrame
def dicts_of_dates(lst_of_dates):
    songs = []
    for i in range(len(lst_of_dates)):
        ranger = list(range(1, 101))
        year = lst_of_dates[i]
        dict_ = {'Position': ranger, 'Artist': split_artists[i], 'Song Title': split_song_titles[i], 'Year': [dates[i] for year in range(1, 101)]}
        as_pandas_df = pd.DataFrame(dict_)
        songs.append(as_pandas_df)
    return songs

songs_16_20 = dicts_of_dates(dates)
all_songs = songs_50_15 + songs_16_20
all_time_billboard_wrap_up = pd.concat(all_songs)
all_time_billboard_wrap_up.reset_index(drop = True, inplace = True) # reset indexing
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950
1,2,Nat King Cole,Mona Lisa,1950
2,3,Anton Karas,Third Man Theme,1950
3,4,Gary and Bing Crosby,Sam's Song,1950
4,5,Gary and Bing Crosby,Simple Melody,1950
...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020
6676,97,Luke Combs,Lovin' On You,2020
6677,98,Moneybagg Yo,Said Sum,2020
6678,99,H.E.R. Featuring YG,Slide,2020


In [6]:
# Pandas dataframe nans checking
print(all_time_billboard_wrap_up['Position'].isna().any())
print(all_time_billboard_wrap_up['Artist'].isna().any())
print(all_time_billboard_wrap_up['Song Title'].isna().any())
print(all_time_billboard_wrap_up['Year'].isna().any())

False
False
False
False


# Genius lyrics

In [7]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# Spliting authors by 'and' and 'Featuring'
artists = list(all_time_billboard_wrap_up.loc[:,"Artist"])
split_artists = []
for artist in artists:
    if any(re.findall(r'and|Featuring', str(artist))):
        result = re.split(r'and|Featuring', str(artist))[0].strip()
        split_artists.append(result)
    else:
        split_artists.append(artist)

all_time_billboard_wrap_up['Split names'] = split_artists
display(all_time_billboard_wrap_up)

,Position,Artist,Song Title,Year,Split names
0,1,Gordon Jenkins and The Weavers,Goodnight Irene,1950,Gordon Jenkins
1,2,Nat King Cole,Mona Lisa,1950,Nat King Cole
2,3,Anton Karas,Third Man Theme,1950,Anton Karas
3,4,Gary and Bing Crosby,Sam's Song,1950,Gary
4,5,Gary and Bing Crosby,Simple Melody,1950,Gary
...,...,...,...,...,...
6675,96,Morgan Wallen,More Than My Hometown,2020,Morgan Wallen
6676,97,Luke Combs,Lovin' On You,2020,Luke Combs
6677,98,Moneybagg Yo,Said Sum,2020,Moneybagg Yo
6678,99,H.E.R. Featuring YG,Slide,2020,H.E.R.


In [28]:
# lyricsgenius settings
genius_token = os.environ.get('genius_token')
genius = lyricsgenius.Genius(genius_token, timeout = 30, retries = 3)
songs = list(all_time_billboard_wrap_up.loc[:, "Song Title"])
artists = list(all_time_billboard_wrap_up.loc[:,"Split names"])

# Saving lyrics to jsons function
def return_lyrics_json(song, artist, json_name):
    song_lyrics = genius.search_song(song, artist)
    with open(json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': song_lyrics.lyrics}, f, ensure_ascii = False, indent = 4)

# No lyrics function (only instrumental songs)
def no_lyrics(json_name):
    with open(json_name, 'w', encoding = 'UTF-8') as f:
        json.dump({'lyrics': ""}, f, ensure_ascii = False, indent = 4)
  

In [29]:
'''
error_list = []
for i in range(len(songs)):
    try:
        return_lyrics_json(songs[i], artists[i], str(i)+'.json')
    except AttributeError:
        no_lyrics(str(i)+'.json')
        error_list.append(i)
'''

"\nerror_list = []\nfor i in range(len(songs)):\n    try:\n        return_lyrics_json(songs[i], artists[i], str(i)+'.json')\n    except AttributeError:\n        no_lyrics(str(i)+'.json')\n        error_list.append(i)\n"

In [27]:
# pd.set_option("display.max_rows", None, "display.max_columns", None)
# err based ong error_list
# Fixed 6140 Beyonce 7/11 (date format in csv)
err = [12, 23, 25, 53, 60, 74, 84, 100, 101, 103, 111, 116, 141, 160, 173, 190, 196, 210, 233, 239, 266, 277, 334, 365, 380, 385, 386, 401, 428, 439, 440, 448, 501, 515, 519, 537, 554, 565, 604, 616, 619, 633, 690, 694, 696, 708, 723, 771, 774, 784, 800, 816, 832, 838, 853, 862, 866, 871, 906, 920, 937, 969, 991, 1039, 1111, 1126, 1150, 1180, 1211, 1245, 1277, 1381, 1387, 1397, 1400, 1422, 1454, 1537, 1542, 1569, 1598, 1613, 1615, 1668, 1801, 1807, 1822, 1850, 1873, 1895, 1940, 1965, 1982, 2027, 2095, 2144, 2167, 2177, 2189, 2208, 2269, 2350, 2370, 2400, 2528, 2559, 2603, 2633, 2690, 2738, 2791, 2835, 2843, 3140, 3334, 3460, 3742, 3908, 3952, 4121, 4169, 4171, 4180, 4208, 4244, 4245, 4280, 4351, 4361, 4387, 4518, 4579, 4760, 4777, 4823, 5134, 5235]
display(all_time_billboard_wrap_up.iloc[err])

,Position,Artist,Song Title,Year,Split names
12,13,Gordon Jenkins and The Weavers,"Tzena, Tzena, Tzena",1950,Gordon Jenkins
23,24,Bill Snyder,Bewitched,1950,Bill Snyder
25,26,Gordon Jenkins,Bewitched,1950,Gordon Jenkins
53,24,Del Wood,Down Yonder,1951,Del Wood
60,1,Leroy Anderson,Blue Tango,1952,Leroy Anderson
74,15,Jo Stafford,Jambalaya,1952,Jo Stafford
84,25,Les Paul,Meet Mr. Callaghan,1952,Les Paul
100,11,Pee Wee Hunt,Oh,1953,Pee Wee Hunt
101,12,Frank Chacksfield,Ebb Tide,1953,Frank Chacksfield
103,14,Richard Hayman,Ruby,1953,Richard Hayman
